In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


### Defining class

In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

- Create instance of neural network and move it to device and print its structure

In [4]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


- Model layers

In [6]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 0.1419,  0.2663,  0.3114,  0.0342,  0.3213,  0.0654,  0.4907,  0.4842,
         -0.5046,  0.4609,  0.5113, -0.1783,  0.0353, -0.1081, -0.0676,  0.5102,
         -0.6900, -0.2031,  0.0335,  0.0592],
        [ 0.1771,  0.2072,  0.4930,  0.3419,  0.4548,  0.1321,  0.4077,  0.2102,
         -0.4628,  0.6402,  0.4574, -0.4335,  0.2624, -0.2098, -0.1585,  0.4121,
         -0.3156, -0.0814, -0.2464,  0.1454],
        [ 0.1600,  0.4635,  0.5538,  0.3598,  0.2651,  0.1552,  0.0862,  0.4199,
         -0.4966,  0.4825,  0.5774, -0.4544,  0.3535,  0.0917, -0.0976, -0.0567,
         -0.4682, -0.0342, -0.1736,  0.1124]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.1419, 0.2663, 0.3114, 0.0342, 0.3213, 0.0654, 0.4907, 0.4842, 0.0000,
         0.4609, 0.5113, 0.0000, 0.0353, 0.0000, 0.0000, 0.5102, 0.0000, 0.0000,
         0.0335, 0.0592],
        [0.1771, 0.2072, 0.4930, 0.3419, 0.4548, 0.1321, 0.4077, 0.2102, 0.0000,
         0.6402, 0.4574, 0.0000, 0.2624, 0.0000, 0.00

In [10]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [12]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0236, -0.0225,  0.0305,  ...,  0.0330, -0.0161,  0.0134],
        [-0.0046,  0.0031, -0.0237,  ..., -0.0043, -0.0308,  0.0165]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0077, -0.0317], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0422,  0.0153, -0.0377,  ..., -0.0205, -0.0418, -0.0207],
        [-0.0324,  0.0007,  0.0258,  ...,  0.0279, -0.0032, -0.0315]],
       device='cuda:0', grad_fn=<Sl